<div class="alert alert-block alert-info">
    <p><b>Курс: </b>Big Data for Data Science</p>
    <p><b>Раздел: </b>3. Apache Spark - задачи</p>
</div>

In [1]:
#   @@@@@@@     @      @    @  @   #
#   @  @  @    @ @      @  @       #
#      @  @   @   @      @@    @   #
#      @     @@@@@@@    @  @   @   #
#      @    @       @  @    @  @   #

Привет, в этой практике мы с вами применим наши знания по PySpark и постараемся изучить что-то новое в процессе выполнения.
<br>В занятии используется датасет собранный на основе данных <a href="https://www.kaggle.com/chicago/chicago-taxi-rides-2016">Chicago Taxi Rides 2016</a>
<br>Полная <a href="https://spark.apache.org/docs/latest/api/python/index.html">документация PySpark</a>.
<br>Схема данны:
<br>|-- taxi_id = идентификатор таксиста
<br>|-- trip_start_timestamp = время начала поездки
<br>|-- trip_end_timestamp = время окончания поездки
<br>|-- trip_seconds = время длительности поездки в секундах
<br>|-- trip_miles = мили проиденные во время поездки
<br>|-- fare = транспортные расходы
<br>|-- tips = назначенные чаевые
<br>|-- trip_total = общая стоимость поездки
<br>|-- payment_type = тип оплаты

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, datediff, avg

In [3]:
spark = SparkSession.builder.appName('PySparkTasks').getOrCreate()

In [4]:
spark.conf.set("spark.sql.session.timeZone", "GMT+3")

In [5]:
spark

Скачайте <a href="https://github.com/AlexKbit/stepik-ds-course/raw/master/Week3/spark-tasks/taxi_data.parquet">taxi_data.parquet</a> и загрузите используя <a href="https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame">SparkAPI</a>

In [6]:
#Ваш код загрузки
df = spark.read.parquet('_data/taxi_data.parquet')

№1 Посчитайте количество загруженных строк.

In [7]:
# Число строк
df.count()

2540712

In [8]:
df.show()

+-------+--------------------+-------------------+------------+----------+-----+-----+----------+------------+
|taxi_id|trip_start_timestamp| trip_end_timestamp|trip_seconds|trip_miles| fare| tips|trip_total|payment_type|
+-------+--------------------+-------------------+------------+----------+-----+-----+----------+------------+
|   5240| 2016-12-15 23:45:00|2016-12-16 00:00:00|         900|       2.5|10.75| 2.45|      14.7| Credit Card|
|   1215| 2016-12-12 07:15:00|2016-12-12 07:15:00|         240|       0.4|  5.0|  3.0|       9.5| Credit Card|
|   3673| 2016-12-16 16:30:00|2016-12-16 17:00:00|        2400|      10.7| 31.0|  0.0|      31.0|        Cash|
|   5400| 2016-12-16 08:45:00|2016-12-16 09:00:00|         300|       0.0| 5.25|  2.0|      7.25| Credit Card|
|   1257| 2016-12-03 18:45:00|2016-12-03 18:45:00|         360|       0.3|  5.0|  0.0|       5.0|        Cash|
|   4666| 2016-12-30 18:00:00|2016-12-30 18:45:00|        2400|      18.2|46.75|10.15|      61.4| Credit Card|
|

Посмотрим схему данных:

In [9]:
df.printSchema()

root
 |-- taxi_id: integer (nullable = true)
 |-- trip_start_timestamp: timestamp (nullable = true)
 |-- trip_end_timestamp: timestamp (nullable = true)
 |-- trip_seconds: integer (nullable = true)
 |-- trip_miles: double (nullable = true)
 |-- fare: double (nullable = true)
 |-- tips: double (nullable = true)
 |-- trip_total: double (nullable = true)
 |-- payment_type: string (nullable = true)



№2 Чему равна корреляция и ковариация между длиной маршрута и ценой за поездку? Ответ округлите до 5 знаков после запятой.
<br>Подробнее <a href="https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame.corr">corr</a> & <a href="https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame.cov">cov</a>

In [10]:
# Ваш код
round(df.corr('trip_miles', 'trip_total'), 5)

0.44816

In [11]:
# Ваш код
round(df.cov('trip_miles', 'trip_total'), 5)

71.96914

№3 Найдите количество, среднее, cреднеквадратическое отклонение, минимум и максимум для длины маршрута и цены за поездку? Ответ округлите до 1 знака после запятой. Подробнее <a href="https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame.describe">describe</a>

In [12]:
# Ваш код
df['trip_miles', 'trip_total'].describe().show()

+-------+------------------+------------------+
|summary|        trip_miles|        trip_total|
+-------+------------------+------------------+
|  count|           2540677|           2540672|
|   mean|3.0005873828090266|15.913560215564042|
| stddev|  5.25716922943536|30.546699217618237|
|    min|               0.0|               0.0|
|    max|             900.0|           9276.69|
+-------+------------------+------------------+



№4 Найдите самый НЕ популярный вид оплаты.
<br>Подробнее <a href="https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame.groupBy">groupBy</a> <a href="https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame.orderBy">orderBy</a>

In [13]:
# Ваш код
payment_types_groupped = df.groupBy('payment_type').count().sort('count', ascending=True).collect()
payment_types_groupped

[Row(payment_type='Way2ride', count=3),
 Row(payment_type='Pcard', count=878),
 Row(payment_type='Prcard', count=968),
 Row(payment_type='Dispute', count=1842),
 Row(payment_type='Unknown', count=5180),
 Row(payment_type='No Charge', count=12843),
 Row(payment_type='Credit Card', count=1108843),
 Row(payment_type='Cash', count=1410155)]

In [14]:
payment_types_groupped[0][0]

'Way2ride'

№5 Найдите идентификатор таксиста выполнившего наибольшее число заказов.

In [15]:
# Ваш код
df.groupBy('taxi_id').count().sort('count', ascending=False).collect()[0][0]

316

№6 Чему равна средняя цена среди поездок, оплаченных наличными? Ответ округлите до 5 знака.
<br> Подробней <a href="https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame.where">where</a>

In [16]:
# Ваш код
cash = df.where(df['payment_type'] == 'Cash').describe()

In [17]:
cash.show()

+-------+-----------------+------------------+------------------+------------------+--------------------+------------------+------------+
|summary|          taxi_id|      trip_seconds|        trip_miles|              fare|                tips|        trip_total|payment_type|
+-------+-----------------+------------------+------------------+------------------+--------------------+------------------+------------+
|  count|          1409697|           1409967|           1410122|           1410119|             1410119|           1410119|     1410155|
|   mean|4360.783044157716| 712.4986067049796|2.2804813271493494|11.194248598877067|0.003325194540318936|12.035261470840307|        null|
| stddev|2516.059908686248|1342.1036613278843| 4.590882504814429| 27.17314538539178|  0.2083038815193268| 36.73317411286132|        null|
|    min|                0|                 0|               0.0|               0.0|                 0.0|               0.0|        Cash|
|    max|             8760|       

№7 Сколько таксистов проехало больше 1000 миль за все время выполнения заказов?

In [18]:
# Ваш код
df_trip_miles = df.groupBy('taxi_id').sum('trip_miles')
df_trip_miles[df_trip_miles['sum(trip_miles)'] > 1000].count()

2860

№8 Сколько миль проехал пассажир в самой долгой поездке? (Ответ округлите до целого)

In [19]:
# Ваш код
df.withColumn('trip_length', datediff(col('trip_end_timestamp'), col('trip_start_timestamp'))).sort('trip_length', ascending=False).show(1)

+-------+--------------------+-------------------+------------+----------+----+----+----------+------------+-----------+
|taxi_id|trip_start_timestamp| trip_end_timestamp|trip_seconds|trip_miles|fare|tips|trip_total|payment_type|trip_length|
+-------+--------------------+-------------------+------------+----------+----+----+----------+------------+-----------+
|    218| 2016-12-05 13:45:00|2016-12-07 12:00:00|        null|       0.0|3.25| 0.0|      3.25|        Cash|          2|
+-------+--------------------+-------------------+------------+----------+----+----+----------+------------+-----------+
only showing top 1 row



№9 Каков средний заработок всех таксистов? Ответ округлите до 5-ого знака.

In [20]:
# Ваш код
df.where(df['taxi_id'].isNotNull())\
    .groupBy('taxi_id')\
    .sum('trip_total')\
    .withColumnRenamed('sum(trip_total)', 'sum_by_taxi')\
    .agg({'sum_by_taxi': 'avg'})\
    .show()

+-----------------+
| avg(sum_by_taxi)|
+-----------------+
|8218.856265256312|
+-----------------+



№10 Сколько поездок начиналось в самый загруженный час?
<br>Используйте функцию <a href="https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.functions.hour">hour</a>

In [21]:
from pyspark.sql.functions import hour

In [22]:
# Ваш код
df.withColumn('hour', hour('trip_start_timestamp')).groupBy('hour').count().sort('count', ascending=False).show(1)

+----+------+
|hour| count|
+----+------+
|  18|181127|
+----+------+
only showing top 1 row



№11 Сколько поездок началось во второй четверти дня?

In [23]:
# Ваш код
df_hour = df.withColumn('hour', hour('trip_start_timestamp')).groupBy('hour').count()
df_hour.where((df_hour['hour'] > 5) & (df_hour['hour'] < 12)).agg({'count': 'sum'}).show()

+----------+
|sum(count)|
+----------+
|    538737|
+----------+



№12 Найдите топ три даты, в которые было суммарно больше всего чаевых? (Чаевые выдаются после совершения поездки)
<br> Ожидаемый формат дат YYYY-MM-DD
<br>Вам может понадобится конвертация типов <a href="https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.Column.cast">cast</a>

In [24]:
from pyspark.sql.types import DateType

In [25]:
# Ваш код
df_date = df.withColumn('date', df['trip_end_timestamp'].cast('date'))
df_date.groupBy('date').sum('tips').sort('sum(tips)', ascending=False).show(3)

+----------+------------------+
|      date|         sum(tips)|
+----------+------------------+
|2016-11-03|110102.37000000013|
|2016-11-09|106187.87999999986|
|2016-11-16| 99993.77000000038|
+----------+------------------+
only showing top 3 rows



№13 Сколько было заказов в дату с наибольшим спросом?

In [26]:
# Ваш код
df_date_2 = df.withColumn('date', df['trip_start_timestamp'].cast('date'))
df_date_2.groupBy('date').count().sort('count', ascending=False).show(1)

+----------+-----+
|      date|count|
+----------+-----+
|2016-11-03|61259|
+----------+-----+
only showing top 1 row



Подгрузите данные о марках машин из датасета <a href="https://github.com/AlexKbit/stepik-ds-course/raw/master/Week3/spark-tasks/taxi_cars_data.parquet">taxi_cars_data.parquet</a>

In [27]:
df_car = spark.read.parquet('_data/taxi_cars_data.parquet')

In [28]:
df_car.show()

+-------+-------------------+
|taxi_id|          car_model|
+-------+-------------------+
|   1159|       Toyota Prius|
|   7273|Ford Crown Victoria|
|   2904|        Honda Civic|
|   3210|        Ford Fusion|
|   2088|       Toyota Camry|
|   4821|Ford Crown Victoria|
|   2069|    Hyundai Elantra|
|   6240|       Toyota Camry|
|    296|     Hyundai Sonata|
|   2136|Ford Crown Victoria|
|   1436|     Toyota Corolla|
|   1090|     Toyota Corolla|
|   7711|        Lincoln MKZ|
|   1572|Ford Crown Victoria|
|   3959|     Chevrolet Aveo|
|   5645|  Chevrolet Lacetti|
|   5149|            Audi A7|
|   6366|     Hyundai Sonata|
|    451|     Hyundai Accent|
|   1669|           Kia Ceed|
+-------+-------------------+
only showing top 20 rows



№14 Какая марка машины самая распрастранненая среди таксистов?
<br>Подробнее <a href="https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.functions.split">split</a>

In [29]:
from pyspark.sql.functions import split

In [30]:
# Ваш код
df_car.withColumn('mark', split(df_car['car_model'], ' ')[0])\
    .groupBy('mark')\
    .count()\
    .sort('count', ascending=False)\
    .show(1)

+----+-----+
|mark|count|
+----+-----+
|Ford| 1484|
+----+-----+
only showing top 1 row



№15 Сколько раз и какая модель машин чаще всего встречается в поездках?
<br>Подробнее <a href="https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame.join">join</a>

In [31]:
# Ваш код
df.join(df_car, on='taxi_id')\
    .groupBy('car_model')\
    .count()\
    .orderBy('count', ascending=False)\
    .show(1)

+-------------------+------+
|          car_model| count|
+-------------------+------+
|Ford Crown Victoria|388682|
+-------------------+------+
only showing top 1 row



Почувствуй силу сжатия! сохрани DataFrame в csv и сравни размеры файлов.

In [32]:
# Ваш код с coalesce(1)
df.coalesce(1).write.csv('taxi_data_csv')

csv    ~ 233 Mb

parquet ~ 29 Mb

Теперь загрузите данные из csv и проверьте типы методом printSchema().

In [33]:
# Ваш код с printSchema() для DataFrame из csv
df_from_csv = spark.read.csv('taxi_data_csv')
df_from_csv.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- _c1: string (nullable = true)
 |-- _c2: string (nullable = true)
 |-- _c3: string (nullable = true)
 |-- _c4: string (nullable = true)
 |-- _c5: string (nullable = true)
 |-- _c6: string (nullable = true)
 |-- _c7: string (nullable = true)
 |-- _c8: string (nullable = true)



In [34]:
df.printSchema()

root
 |-- taxi_id: integer (nullable = true)
 |-- trip_start_timestamp: timestamp (nullable = true)
 |-- trip_end_timestamp: timestamp (nullable = true)
 |-- trip_seconds: integer (nullable = true)
 |-- trip_miles: double (nullable = true)
 |-- fare: double (nullable = true)
 |-- tips: double (nullable = true)
 |-- trip_total: double (nullable = true)
 |-- payment_type: string (nullable = true)



Не забудьте посетить SparkUI и изучить историю ваших задач.